# NeuralNetworks

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from collections import defaultdict

## Data Preparation

In [ ]:
X_train = pd.read_csv("X_train.txt", header=None,  delim_whitespace=True)
y_train = pd.read_csv("y_train.txt", header=None, delim_whitespace=True )
X_test = pd.read_csv("X_test.txt", header=None, delim_whitespace=True )
y_test = pd.read_csv("y_test.txt", header=None, delim_whitespace=True )
#subject_test = pd.read_csv("subject_test.txt", header=None, delim_whitespace=True )
subject_train = pd.read_csv("subject_train.txt", header=None, delim_whitespace=True )
features = pd.read_csv("features.txt", header=None, delim_whitespace=True )

In [ ]:
feature = features
feature.drop(0,inplace=True, axis=1)

In [ ]:
#inseriamo l'intestazione al dataset
lista=[]
feat_transpa = feature.transpose()
for i in range(561):
    lista.append(feat_transpa.iloc[0][i])
X_test.columns=lista    
X_train.columns=lista

In [ ]:
# eliminiamo le colonne che contengono la stima mad in quanto quasi uguale a dev.std
stringa="mad()"
for col in X_train.columns:
    if(stringa in col):
        X_train.drop(labels=col, axis=1, inplace=True)
for col in X_test.columns:
    if(stringa in col):
        X_test.drop(labels=col, axis=1, inplace=True)
#for col in features:
#    if (stringa in col):
#        features.drop(labels=col,axis=1,inplace=True)

## Sklearn Multilayer Perceptron

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
hidden_layer_sizes = (100, 200,)

### Parameters

hidden_layer_sizes tuple, length = n_layers - 2, default=(100,)
The ith element represents the number of neurons in the ith hidden layer.

activation {'identity', 'logistic', 'tanh', 'relu'}, default='relu'
Activation function for the hidden layer.
* 'identity', no-op activation, useful to implement linear bottleneck, returns f(x) = x
* 'logistic', the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).
* 'tanh', the hyperbolic tan function, returns f(x) = tanh(x).
* 'relu', the rectified linear unit function, returns f(x) = max(0, x)

solver {'lbfgs', 'sgd', 'adam'}, default='adam'
The solver for weight optimization.
* 'lbfgs' is an optimizer in the family of quasi-Newton methods.
* 'sgd' refers to stochastic gradient descent.
* 'adam' refers to a stochastic gradient-based optimizer proposed by Kingma, Diederik, and Jimmy Ba
The default solver 'adam' works pretty well on relatively large datasets (>= 1000 training samples) in terms of both training time and validation score. For small datasets, 'lbfgs' can converge faster and perform better.

alpha float, default=0.0001
L2 penalty (regularization term) parameter.

batch_size int, default='auto'
Size of minibatches for stochastic optimizers. If the solver is 'lbfgs', the classifier will not use minibatch. When set to “auto”, batch_size=min(200, n_samples)

learning_rate {'constant', 'invscaling', 'adaptive'}, default='constant'
Learning rate schedule for weight updates.
*'constant' is a constant learning rate given by 'learning_rate_init'.
*'invscaling' gradually decreases the learning rate at each time step 't' using an inverse scaling exponent of *'power_t'. effective_learning_rate = learning_rate_init / pow(t, power_t)
*'adaptive' keeps the learning rate constant to 'learning_rate_init' as long as training loss keeps decreasing. Each time two consecutive epochs fail to decrease training loss by at least tol, or fail to increase validation score by at least tol if 'early_stopping' is on, the current learning rate is divided by 5.
Only used when solver='sgd'.

learning_rate_init double, default=0.001
The initial learning rate used. It controls the step-size in updating the weights. Only used when solver='sgd' or 'adam'.

power_t double, default=0.5
The exponent for inverse scaling learning rate. It is used in updating effective learning rate when the learning_rate is set to 'invscaling'. Only used when solver='sgd'.

max_iter int, default=200
Maximum number of iterations. The solver iterates until convergence (determined by 'tol') or this number of iterations. For stochastic solvers ('sgd', 'adam'), note that this determines the number of epochs (how many times each data point will be used), not the number of gradient steps.

shuffle bool, default=True
Whether to shuffle samples in each iteration. Only used when solver='sgd' or 'adam'.

random_state int, RandomState instance or None, default=None
If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator; If None, the random number generator is the RandomState instance used by np.random.

tol float, default=1e-4
Tolerance for the optimization. When the loss or score is not improving by at least tol for n_iter_no_change consecutive iterations, unless learning_rate is set to 'adaptive', convergence is considered to be reached and training stops.

verbose bool, default=False
Whether to print progress messages to stdout.

warm_start bool, default=False
When set to True, reuse the solution of the previous call to fit as initialization, otherwise, just erase the previous solution. See the Glossary.

momentum float, default=0.9
Momentum for gradient descent update. Should be between 0 and 1. Only used when solver='sgd'.

early_stopping bool, default=False
Whether to use early stopping to terminate training when validation score is not improving. If set to true, it will automatically set aside 10% of training data as validation and terminate training when validation score is not improving by at least tol for n_iter_no_change consecutive epochs. The split is stratified, except in a multilabel setting. Only effective when solver='sgd' or 'adam'

validation_fraction float, default=0.1
The proportion of training data to set aside as validation set for early stopping. Must be between 0 and 1. Only used if early_stopping is True

beta_1 float, default=0.9
Exponential decay rate for estimates of first moment vector in adam, should be in [0, 1). Only used when solver='adam'

beta_2 float, default=0.999
Exponential decay rate for estimates of second moment vector in adam, should be in [0, 1). Only used when solver='adam'

epsilon float, default=1e-8
Value for numerical stability in adam. Only used when solver='adam'

n_iter_no_change int, default=10
Maximum number of epochs to not meet tol improvement. Only effective when solver='sgd' or 'adam'


### Attributes
loss_ float
The current loss computed with the loss function.

coefs_ list, length n_layers - 1
The ith element in the list represents the weight matrix corresponding to layer i.

intercepts_ list, length n_layers - 1
The ith element in the list represents the bias vector corresponding to layer i + 1.

n_iter_ int,
The number of iterations the solver has ran.

n_layers_ int
Number of layers.

n_outputs_ int
Number of outputs.

out_activation_ string
Name of the output activation function.

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(128, 64, 32,), alpha=0.1, learning_rate='adaptive', 
                    activation='tanh', early_stopping=False, momentum=0.9, random_state=0)

clf.fit(X_train,  np.ravel(y_train))

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

In [ ]:
#Loss curve
plt.plot(clf.loss_curve_)
plt.show()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

parameter = {
    'hidden_layer_sizes': [(32,64), (128,64,32),  (150,50), (100)],
    'activation': ['tanh', 'relu', 'logistic'],
    'solver': ['sgd', 'adam'],
    #'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

    #'hidden_layer_sizes': [(50,50), (100,100), (100,150), (150,50), (100,50)],


clf = GridSearchCV(MLPClassifier(), parameter, n_jobs=-1, cv=3)
clf.fit(X_train, np.ravel(y_train))

In [ ]:
y_pred = clf.predict(X_test)
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(32,64),  learning_rate='constant', 
                    activation='logistic', solver= 'adam', momentum=1, batch_size=64, random_state=0)

clf.fit(X_train,  np.ravel(y_train))

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

## Keras Deep Neural Networks

In [ ]:
#!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
def build_model():
    
    n_feature = len(X_train.columns)
    model = Sequential()
    model.add(Dense(64, input_dim=n_feature, activation='sigmoid'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(72, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(80, activation='sigmoid'))
    model.add(Dense(96, activation='relu'))
    model.add(Dense(7, activation='softmax'))

    

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model1 = build_model()
history1 = model1.fit(X_train, y_train, epochs=1000, batch_size=50).history

In [ ]:
model2 = build_model()
history2 = model2.fit(X_train, y_train, epochs=500, batch_size=50).history

In [ ]:
#Cross-entropy vs Epochs with different batch_size
plt.plot(history1['loss'], label='Loss -> batch 50')
plt.plot(history2['loss'], label='Loss -> batch 150')
plt.xlabel('Epochs')
plt.ylabel('Cross-Entropy')
plt.legend()
plt.show()

In [ ]:
#Evaluate models
test_loss_1, test_acc_1 = model1.evaluate(X_test, y_test)
test_loss_2, test_acc_2 = model2.evaluate(X_test, y_test)

print('Loss %f, Accuracy %f' % (test_loss_1, test_acc_1))
print('Loss %f, Accuracy %f' % (test_loss_2, test_acc_2))

In [ ]:
y_pred = model1.predict(X_test).astype(int)
y_pred
#print('Accuracy %s' % accuracy_score(y_test, y_pred))
#print('F1-score %s' % f1_score(y_test, y_pred, average=None))
#print(classification_report(y_test, y_pred))

In [ ]:
#Split to fit the model and perform it on 20% of training data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

model3 = build_model()
history3 = model3.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=300, batch_size=50).history

In [ ]:
test_loss_3, test_acc_3 = model3.evaluate(X_test, y_test)

print('Loss %f, Accuracy %f' % (test_loss_1, test_acc_1))
print('Loss %f, Accuracy %f' % (test_loss_2, test_acc_2))
print('Loss %f, Accuracy %f' % (test_loss_3, test_acc_3))

In [ ]:
#Cross_Entropy Train & Validation Set
plt.plot(history3['loss'], label='Train')
plt.plot(history3['val_loss'], label='Val')
plt.xlabel('Epochs')
plt.ylabel('Cross-Entropy')
plt.legend()
plt.show()

## Early Stopping with ModelCheckpoint

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

The most relevant arguments are:

* monitor: quantity to be monitored
* patience: number of epochs with no improvement after which training will be stopped

In [ ]:
es = EarlyStopping(monitor='val_loss', patience=10)
mc = ModelCheckpoint('best_model_NOREG.h5', monitor='val_loss', save_best_only=True)

model4 = build_model()
history4 = model4.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, 
                      batch_size=10, callbacks=[es,mc]).history

In [ ]:
test_loss_4, test_acc_4 = model4.evaluate(X_test, y_test)

print('Loss %f, Accuracy %f' % (test_loss_1, test_acc_1))
print('Loss %f, Accuracy %f' % (test_loss_2, test_acc_2))
print('Loss %f, Accuracy %f' % (test_loss_3, test_acc_3))
print('Loss %f, Accuracy %f' % (test_loss_4, test_acc_4))

### L2 Regularization

In [ ]:
from keras.regularizers import l2

def build_L2_model():
    # define the model
    model = Sequential()

    n_feature = X_train.shape[1]
    h_dim=100
    model.add(Dense(h_dim, activation='relu', input_shape=(n_feature,), kernel_regularizer=l2(0.01)))
    model.add(Dense(h_dim, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dense(h_dim, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dense(h_dim, activation='relu', kernel_regularizer=l2(0.01)))
    #linear activation
    model.add(Dense(1))

    #compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
mc = ModelCheckpoint('best_model_L2.h5', monitor='val_loss', save_best_only=True)

L2_model = build_L2_model()
h_L2 = L2_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, 
                    batch_size=10, callbacks=[es,mc]).history

### Dropout

In [ ]:
from keras.layers import Dropout

def build_DROPOUT_model():
    # define the model
    model = Sequential()

    n_feature = X_train.shape[1]
    h_dim=100
    model.add(Dense(h_dim, activation='relu', input_shape=(n_feature,)))
    model.add(Dropout(0.2))
    model.add(Dense(h_dim, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(h_dim, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(h_dim, activation='relu'))
    #linear activation
    model.add(Dense(1))

    #compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
mc = ModelCheckpoint('best_model_DROPOUT.h5', monitor='val_loss', save_best_only=True)

DROPOUT_model = build_DROPOUT_model()
h_DROPOUT = DROPOUT_model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                              epochs=100, batch_size=10, callbacks=[es,mc]).history

In [ ]:
# laod best models and test them
from keras.models import load_model

best_NOREG_model = load_model('best_model_NOREG.h5')
best_L2_model = load_model('best_model_L2.h5')
best_DROPOUT_model = load_model('best_model_DROPOUT.h5')

loss_NOREG, acc_NOREG = best_NOREG_model.evaluate(X_test, y_test)
loss_L2, acc_L2 = best_L2_model.evaluate(X_test, y_test)
loss_DROPOUT, acc_DROPOUT = best_DROPOUT_model.evaluate(X_test, y_test)

In [ ]:
print('Loss %f, Accuracy %f' % (loss_NOREG, acc_NOREG))
print('Loss %f, Accuracy %f' % (loss_L2, acc_L2))
print('Loss %f, Accuracy %f' % (loss_DROPOUT, acc_DROPOUT))

## Deep Neural network with Hyperparametertuning

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
def build_model1(hp):
    
    model = keras.Sequential()
    n_feature = X_train.shape[1]    
    #model.add(tf.keras.layers.Dense(8, activation='tanh',input_shape=(n_feature,)))
    
    for i in range(hp.Int('layers',2,6)):
        if (i==0):
            model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i), 8, 128, step=8), 
            activation=hp.Choice('act_'+ str(i), ['relu', 'tanh','sigmoid']),input_shape=(n_feature,)))
        else:
            model.add(tf.keras.layers.Dense( 
                units=hp.Int('units_' + str(i), 8, 128, step=8), 
                activation=hp.Choice('act_'+ str(i), ['relu', 'tanh','sigmoid'])))
    
    #NB Impostare numero di neuroni ultimo layer = N_class + 1
    model.add(tf.keras.layers.Dense(7, activation='softmax'))
    model.compile('adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
#from kerastuner.tuners import RandomSearch
tuner = RandomSearch(
    build_model1,
    objective='val_accuracy', #massimizziamo la validation accuracy
    max_trials=40,
    executions_per_trial=2,
    directory='prove',
    project_name='final_ultimatex')

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(X_train,y_train,
            epochs=40,
            validation_split=0.1)

In [ ]:
tuner.results_summary()

In [ ]:
# Retrieve the best model.
best_model = tuner.get_best_models(num_models=1)[0]
best_model

In [ ]:
#Split to fit the model and perform it on 20% of training data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)

In [ ]:
history1 = best_model.fit(X_train,y_train, epochs=200, batch_size=10).history #, validation_data=(X_val,y_val)

In [ ]:
history2 = best_model.fit(X_train, y_train, epochs=200, batch_size=50).history

In [ ]:
#Cross_entropy with different Batch_size
plt.plot(history['loss'], label='Loss 10')
plt.plot(history2['loss'], label='Loss 50')

plt.xlabel('Epochs')
plt.ylabel('Cross-Entropy')
plt.legend()
plt.show()